# Homework Week 11, Heather Leighton-Dick

## 1.	What scale of measurement applies to timeseries data? Is timeseries data continuous or discrete? Why?

Timeseries data is quantitative and is measured on an interval scale with the same distance between data points. Timeseries data is continuous because the distances between data points can be divided into smaller increments.

## 2.	What does ARIMA stand for? Break down and explain each piece in the name.

ARIMA stands for Auto Regression Integration Moving Average. AR (autoregressive) is a model that uses input from previous steps in time for calculating present values and predicting future values. (Order of) Integration is a process used to make a series stationary (eliminates exponential . The Moving Average model smoothes out data over a particular time period by creating a constantly averaged value.

## 3.	Use the data in the temperatures/ folder and write a function that efficiently converts every column to a different temperature scale; one that is more common than Kelvin.

In [98]:
import numpy as np
import pandas as pd
import math
import statistics
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson

from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
#from tqdm import tqdm_notebook
#from itertools import product
import warnings
warnings.filterwarnings('ignore')

#bring in csv and convert it to a dataframe
temperature_path = "temperature.csv"
temperature_df = pd.read_csv(temperature_path)
temperature_df = pd.DataFrame(temperature_df)
print(temperature_df.head(5))
print(temperature_df.tail(5))
print(len(temperature_df))
print(temperature_df["Seattle"])
#print(temperature_df.info())
#print(temperature_df.describe())
#print(temperature_df.index)

          datetime   Vancouver    Portland  San Francisco     Seattle  \
0  10/1/2012 12:00         NaN         NaN            NaN         NaN   
1  10/1/2012 13:00  284.630000  282.080000     289.480000  281.800000   
2  10/1/2012 14:00  284.629041  282.083252     289.474993  281.797217   
3  10/1/2012 15:00  284.626998  282.091866     289.460618  281.789833   
4  10/1/2012 16:00  284.624955  282.100481     289.446243  281.782449   

   Los Angeles   San Diego   Las Vegas     Phoenix  Albuquerque  ...  \
0          NaN         NaN         NaN         NaN          NaN  ...   
1   291.870000  291.530000  293.410000  296.600000   285.120000  ...   
2   291.868185  291.533501  293.403141  296.608509   285.154558  ...   
3   291.862844  291.543355  293.392177  296.631487   285.233952  ...   
4   291.857503  291.553209  293.381213  296.654466   285.313345  ...   

   Philadelphia    New York    Montreal      Boston   Beersheba  \
0           NaN         NaN         NaN         NaN         N

In [108]:
#convert Kelvins to Fahrenheit in each city column, avoiding NaNs
#dates span from 10-1-2012 to 11-30-2017

def convert_K_to_F(x):    
    if type(x) == str:
        return x
    else:
        return ((x - 273.15) * 1.8) + 32

temperatureF_df = temperature_df.applymap(convert_K_to_F, na_action="ignore")
print(temperatureF_df.head(10))

          datetime  Vancouver   Portland  San Francisco    Seattle  \
0  10/1/2012 12:00        NaN        NaN            NaN        NaN   
1  10/1/2012 13:00  52.664000  48.074000      61.394000  47.570000   
2  10/1/2012 14:00  52.662274  48.079854      61.384987  47.564990   
3  10/1/2012 15:00  52.658596  48.095360      61.359113  47.551699   
4  10/1/2012 16:00  52.654918  48.110866      61.333238  47.538407   
5  10/1/2012 17:00  52.651240  48.126372      61.307364  47.525116   
6  10/1/2012 18:00  52.647562  48.141878      61.281489  47.511825   
7  10/1/2012 19:00  52.643884  48.157384      61.255615  47.498534   
8  10/1/2012 20:00  52.640206  48.172890      61.229740  47.485243   
9  10/1/2012 21:00  52.636528  48.188396      61.203866  47.471951   

   Los Angeles  San Diego  Las Vegas    Phoenix  Albuquerque  ...  \
0          NaN        NaN        NaN        NaN          NaN  ...   
1    65.696000  65.084000  68.468000  74.210000    53.546000  ...   
2    65.692734  65.090

In [20]:
def x(a):
    return a + 1 

df_new  = df.apply(x, axis = 1)

print("The original dataframe:")
print(df)
print("The new dataframe:")
print(df_new)



def convert_temp(scale=None, source_temp=None):
    if scale == "F":
        return 'C', (source_temp - 32.0) * (5.0/9.0)
    elif scale == "C":
        return 'F', (source_temp * (9.0/5.0)) + 32.0
    else:
        print("Needs to be (F) or (C)!")
        
temp = input("Input the  temperature you like to convert? (e.g., 45F, 102C etc.) : ")
degree = int(temp[:-1])
i_convention = temp[-1]

if i_convention.upper() == "C":
  result = int(round((9 * degree) / 5 + 32))
  o_convention = "Fahrenheit"
elif i_convention.upper() == "F":
  result = int(round((degree - 32) * 5 / 9))
  o_convention = "Celsius"
else:
  print("Input proper convention.")
  quit()
print("The temperature in", o_convention, "is", result, "degrees.")
--------------------------------------
df['B'] = df['A'].replace({'M' : '*10000', 'K' : '*1'},regex=True).map(pd.eval)
--------------------------------------
def fahr_to_celsius(temp_fahr):
    """Convert Fahrenheit to Celsius
    
    Return Celsius conversion of input"""
    temp_celsius = (temp_fahr - 32) * 5 / 9
    return temp_celsius


fahr_to_celsius(data["temperature"])

#https://sparkbyexamples.com/pandas/pandas-apply-function-usage-examples/


SyntaxError: invalid syntax (2232600446.py, line 23)

## 4.	Write a function that efficiently fills in missing values in every column, excluding trailing NANs. That is: if there's temperature data on either side of a timestamp, it should interpolate through it. But if data stopped being collected, your function should leave NaNs at the end.
sdff

In [ ]:
#fill in missing values with O's except for trailing NaNs
def fill_in(x):
    if x is alone, leave it
    if x is surrounded, change it
    df.dropna() except if NaN is at end of column

## 5.	Find all cities A and B where city A is warmer than city B in the summer, but also colder than city B in the winter, on average.

In [ ]:
#take average of city A in summer and winter, and average of city B in summer and winter
#compare city A summer > city B summer AND city A winter < city B winter

#for each city, find the average temperature for the summer (J J A), 
#then take average of that season across all years

## 6.	For two of the cities you found in question 5 (there are many possible answers), clean up the data via rolling averages and plot a clear visual. Explain how rolling average differs from moving average (MA) model.

## 7.	Perform SARIMA on five cities, iterating through the cities programmatically. Explain your results.

In [12]:
#https://towardsdatascience.com/time-series-forecasting-with-sarima-in-python-cda5b793977b
adfuller_Seattle = adfuller(temperature_df['Seattle'])
print(adfuller_Seattle[1])

MissingDataError: exog contains inf or nans

## 7a.	Graph the predicted values 30 days out.

## 8.	Explain what you see when you perform results.summary() on the cities in question 7. 

## Bonus:
What do GARCH and ARCH Stand for? Break down and explain each piece in the names. Explain the difference between the two. Create a GARCH and ARCH model on the same 5 cities from question 6. Explain and graph the results.
